Practicing linear algebra

In [1]:
import numpy as np

Gaussian elemination and pivots

In [11]:
def gaussian_elemination(A, t=1e-9):
    A = A.astype(float).copy()
    m, n = A.shape
    row = 0
    pivots = []

    for col in range(n):
        if row >= m:
            break

        pivot_row = None
        max_val = t
        for r in range(row, m):
            if abs(A[r, col]) > max_val:
                max_val = abs(A[r, col])
                pivot_row = r

        if pivot_row is None:
            continue

        if pivot_row != row:
            A[[row, pivot_row]] = A[[pivot_row, row]]

        pivot_val = A[row, col]
        A[row, :] = A[row, :] / pivot_val

        for r in range(row + 1, m):
            factor = A[r, col]
            A[r, :] = A[r, :] - factor * A[row, :]

        # ONLY store pivot column index
        pivots.append(col)
        row += 1

    return A, pivots

A=np.array([[2,1,-1,8],
            [-3,-1,2,-11],
            [-2,1,2,-3]])

A,pivots=gaussian_elemination(A)
print("Row Echelon Form:\n",A ,"\nPivots:",pivots)




Row Echelon Form:
 [[ 1.          0.33333333 -0.66666667  3.66666667]
 [ 0.          1.          0.4         2.6       ]
 [ 0.          0.          1.         -1.        ]] 
Pivots: [0, 1, 2]


NULL SPACE

In [22]:
def null_space_basis(A, tol=1e-9):
   
    m, n = A.shape
    R, pivots = gaussian_elemination(A, t=tol)

    pivot_set = set(pivots)
    free_vars = [j for j in range(n) if j not in pivot_set]

    basis = []

    for free in free_vars:
        x = np.zeros(n)
        x[free] = 1.0

        for i in range(len(pivots) - 1, -1, -1):  
            col = pivots[i]                      
            s = 0.0
            for j in range(col + 1, n):
                s += R[i, j] * x[j]
            x[col] = -s    
        basis.append(x)

    return basis
A = np.array([[1, 2, 3, 4],
              [2, 4, 6, 8],
              [3, 6, 9, 12]])
basis = null_space_basis(A)
print("Basis for Null Space:",'\n' ,basis)

Basis for Null Space: 
 [array([-2.,  1.,  0.,  0.]), array([-3.,  0.,  1.,  0.]), array([-4.,  0.,  0.,  1.])]


For Ax=b

In [16]:

def solve_Ax_b(A, b, tol=1e-9):
    
    A = A.astype(float)
    b = b.astype(float).reshape(-1, 1)
    m, n = A.shape

    # Build augmented matrix [A | b]
    Ab = np.hstack([A, b])

    # REF on augmented system
    R, pivots = gaussian_elemination(Ab)
    R_A = R[:, :n]   # left part (transformed A)
    R_b = R[:, n]    # right part (transformed b)

    # 1) Check consistency:
    # if a row is [0,0,...,0 | non-zero] => no solution
    for i in range(m):
        if np.all(np.abs(R_A[i, :]) < tol) and abs(R_b[i]) > tol:
            # Inconsistent system
            return None, None

    # 2) Null space basis of A (solutions of Ax=0)
    basis_null = null_space_basis(A, tol=tol)

    # 3) Find one particular solution by setting all free vars = 0
    x = np.zeros(n)

    # Back substitution for particular solution using R_A x = R_b
    for i in range(m - 1, -1, -1):   # from bottom row upwards
        # find pivot column in row i
        pivot_col = None
        for j in range(n):
            if abs(R_A[i, j]) > tol:
                pivot_col = j
                break

        if pivot_col is None:
            continue  # zero row, skip

        # Row looks like: x_pivot + sum_{j>pivot} R_A[i,j]*x_j = R_b[i]
        s = 0.0
        for j in range(pivot_col + 1, n):
            s += R_A[i, j] * x[j]
        x[pivot_col] = R_b[i] - s

    particular = x
    return particular, basis_null

A=np.array([[2,1,-1],
            [-3,-1,2],  
            [-2,1,2]])
b=np.array([8,-11,-3])
particular, basis_null = solve_Ax_b(A,b)    
print("Particular Solution:",particular)
print("Basis for Null Space:",basis_null)



Particular Solution: [ 2.  3. -1.]
Basis for Null Space: []
